In [1]:
# import warnings
# warnings.filterwarnings('ignore')

In [2]:
### Установим красивые дефолтные настройки
### Может быть лень постоянно прописывать
### У графиков параметры цвета, размера, шрифта
### Можно положить их в словарь дефолтных настроек

import matplotlib as mlp

mlp.rcParams['lines.linewidth'] = 5
mlp.rcParams['xtick.major.size'] = 20
mlp.rcParams['xtick.major.width'] = 5
mlp.rcParams['xtick.labelsize'] = 20
mlp.rcParams['xtick.color'] = '#FF5533'

mlp.rcParams['ytick.major.size'] = 20
mlp.rcParams['ytick.major.width'] = 5
mlp.rcParams['ytick.labelsize'] = 20
mlp.rcParams['ytick.color'] = '#FF5533'

mlp.rcParams['axes.labelsize'] = 20
mlp.rcParams['axes.titlesize'] = 20
mlp.rcParams['axes.titlecolor'] = '#00B050'
mlp.rcParams['axes.labelcolor'] = '#00B050'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime



pd.options.display.max_columns = 500

In [4]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, TimeSeriesSplit, train_test_split, RandomizedSearchCV
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
# from category_encoders import TargetEncoder
# from category_encoders.one_hot import OneHotEncoder

## Обработка данных

In [5]:
feed_data = pd.read_csv("feed_data.csv")
feed_data.head()

,timestamp,user_id,post_id,action,target
0,2021-11-26 15:50:14,200,1881,view,1
1,2021-12-13 08:55:58,200,1452,view,1
2,2021-11-30 09:37:28,200,249,view,1
3,2021-12-13 08:14:54,200,1198,view,1
4,2021-11-20 22:45:25,200,6884,view,1


In [6]:
feed_data.shape

(4605687, 5)

In [7]:
pd.isnull(feed_data).sum()

timestamp    0
user_id      0
post_id      0
action       0
target       0
dtype: int64

In [8]:
post_text_df = pd.read_csv("post_text_with_embeddings.csv", sep=';')
post_text_df.head()

,post_id,text,topic,embeddings
0,1,uk economy facing major risks the uk manufactu...,business,"[-0.02879844792187214, 0.018772652372717857, -..."
1,2,aids and climate top davos agenda climate chan...,business,"[0.0011228763032704592, 0.04378493130207062, -..."
2,3,asian quake hits european shares shares in eur...,business,"[-0.0488189235329628, 0.05721372738480568, 0.0..."
3,4,india power shares jump on debut shares in ind...,business,"[-0.024175573140382767, 0.021884476765990257, ..."
4,5,lacroix label bought by us firm luxury goods g...,business,"[-0.03241302818059921, 0.05497121810913086, -0..."


In [9]:
pd.isnull(post_text_df).sum()

post_id       0
text          0
topic         0
embeddings    0
dtype: int64

In [10]:
post_text_df.shape

(7023, 4)

In [11]:
user_data = pd.read_csv("user_data.csv")

In [12]:
user_data.shape

(163205, 8)

In [13]:
user_data.head()

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


## Преобразование данных

### Обработаем колонку feed_data

In [14]:
# Отношение количества строк с target = 0 к количеству строк с target = 1 
feed_data['target'].value_counts()[0] / feed_data['target'].value_counts()[1]

0.9340752971648711

In [15]:
# удаление дублирующих данных
feed_data = feed_data.iloc[feed_data.drop('timestamp', axis=1).drop_duplicates().index]

# преобразуем тип клонки timestamp в datetime
feed_data['timestamp'] = pd.to_datetime(feed_data['timestamp'])

# преобразуем время в секунды
# feed_data['timestamp_sec'] =(feed_data['timestamp']-datetime.datetime(2021, 9, 1)).dt.total_seconds()

# выделим час и месяц когда смотрелся 
feed_data['hour'] = feed_data['timestamp'].dt.hour
feed_data['month'] = feed_data['timestamp'].dt.month

# Удалим колонку 'timestamp'
# feed_data = feed_data.drop('timestamp', axis=1)

# исключим позиции у которых action = like

feed_data = feed_data[feed_data['action'] != 'like']

# исключим фичу action, т.к. у нее одно значение

feed_data = feed_data.drop('action', axis=1)

In [16]:
feed_data.head()

,timestamp,user_id,post_id,target,hour,month
0,2021-11-26 15:50:14,200,1881,1,15,11
1,2021-12-13 08:55:58,200,1452,1,8,12
2,2021-11-30 09:37:28,200,249,1,9,11
3,2021-12-13 08:14:54,200,1198,1,8,12
4,2021-11-20 22:45:25,200,6884,1,22,11


In [17]:
feed_data.shape

(4593428, 6)

### Обработаем колонку post_text

In [18]:
post_text_df.head(-1)

,post_id,text,topic,embeddings
0,1,uk economy facing major risks the uk manufactu...,business,"[-0.02879844792187214, 0.018772652372717857, -..."
1,2,aids and climate top davos agenda climate chan...,business,"[0.0011228763032704592, 0.04378493130207062, -..."
2,3,asian quake hits european shares shares in eur...,business,"[-0.0488189235329628, 0.05721372738480568, 0.0..."
3,4,india power shares jump on debut shares in ind...,business,"[-0.024175573140382767, 0.021884476765990257, ..."
4,5,lacroix label bought by us firm luxury goods g...,business,"[-0.03241302818059921, 0.05497121810913086, -0..."
...,...,...,...,...
7017,7314,although the likeliness of someone focusing on...,movie,"[-0.016084404662251472, 0.011012321338057518, ..."
7018,7315,ok i would not normally watch a farrelly brot...,movie,"[-0.05603279173374176, 0.013912963680922985, -..."
7019,7316,i give this movie 2 stars purely because of it...,movie,"[-0.05724164471030235, 0.049697499722242355, -..."
7020,7317,i cant believe this film was allowed to be mad...,movie,"[-0.026836182922124863, 0.020415475592017174, ..."


In [19]:
def embeddings_to_list(x):
    x = x.replace('[', '').replace(']', '').split(',')
    x = list(map(lambda x: float(x), x))
    return x
post_text_df['embeddings'] = post_text_df['embeddings'].apply(embeddings_to_list)

In [20]:
# выделим эмбединги

In [21]:
post_text_embedings = post_text_df.drop(['topic','text'], axis=1)

In [22]:
post_text_embedings = pd.concat([post_text_embedings.drop(['embeddings'], axis=1), pd.DataFrame(post_text_embedings['embeddings'].to_list())], axis=1)

In [23]:
post_text_embedings.head(-1)

,post_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,...,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,1,-0.028798,0.018773,-0.001214,-0.125246,0.039150,-0.076206,0.000964,-0.079195,0.124329,-0.007496,-0.092711,0.187947,0.003903,0.095597,0.049756,-0.071081,0.063811,0.079639,0.032890,-0.156823,-0.140350,0.096549,-0.073233,0.116392,0.100085,-0.070297,0.079477,0.127897,-0.142257,-0.009965,-0.015595,-0.005024,-0.008289,0.038013,0.080437,0.102658,0.104131,-0.048555,0.017346,0.053889,0.079000,-0.020050,0.002036,0.016203,0.034028,0.008022,0.030125,0.119203,-0.030215,0.023926,-0.004007,0.060271,-0.024972,0.023055,-0.086985,-0.043570,-0.028596,0.063366,-0.013393,-0.056809,0.053876,-0.201463,-0.076844,0.021904,-0.007468,-0.014996,-0.042449,-0.136527,0.002443,0.084681,0.125636,-0.022920,0.083131,-0.013225,-0.068850,0.032893,0.039834,0.199923,-0.112005,-0.066690,0.116589,0.065965,0.258591,0.000208,0.059341,-0.025162,0.021798,-0.014108,0.051381,0.052344,0.043631,-0.012191,-0.067974,0.094771,0.071056,0.059150,-0.017749,-0.058418,-0.038238,-0.112956,-0.018238,-0.080328,0.006399,0.078605,-0.028007,0.071577,-0.024571,-0.019803,0.088965,-0.052238,0.097489,0.108962,0.051023,0.052053,0.014231,0.001908,-0.027284,0.064806,0.056422,-0.037387,0.035440,0.007560,0.158067,-0.020930,0.044062,0.051756,-0.086708,0.051404,-0.079033,0.046839,-0.037015,-0.097859,-0.002108,0.057318,0.022282,0.071768,-0.005628,0.000290,0.096259,0.002869,-0.019507,0.072069,0.048805,-0.042219,0.210244,0.013346,0.007545,-0.098892,0.021108,-0.045255,0.138777,-0.094706,-0.070853,0.026436,-0.056051,0.490795,0.064099,0.082167,-0.056069,-0.020802,0.183870,0.072214,0.021592,-0.012569,-0.035191,-0.011287,-0.032689,0.089632,0.059130,-0.128141,0.040040,0.007326,0.021590,-0.003479,-0.076959,0.002330,-0.000252,-0.014421,-0.056529,-0.008707,0.089488,0.095467,-0.049604,0.043725,-0.047477,0.073911,0.015861,0.115352,0.040713,0.012682,0.028292,-0.047336,0.006179,-0.019181,-0.031696,0.030778,-0.004626,-0.023711,0.052489,-0.019655,0.027952,-0.055492,0.075395,-0.067975,0.015591,0.016978,0.043507,0.009332,0.026375,-0.025187,-0.043135,-0.000607,0.006968,0.067409,-0.079372,-0.020252,0.042080,0.263691,-0.048578,0.051441,0.004870,0.002713,-0

In [24]:
# выделим основные фичи: среднее, минимум, максимум

In [25]:
post_text_embedings.drop(['post_id'], axis=1).max(axis=1)

0       10.335843
1       10.597654
2       10.482660
3       10.398684
4       10.584318
          ...    
7018    11.109606
7019    11.090212
7020    10.406042
7021    11.186502
7022    11.148056
Length: 7023, dtype: float64

In [26]:
post_text_df['EmbMean'] = post_text_embedings.drop(['post_id'], axis=1).mean(axis=1)
post_text_df['EmbSum'] = post_text_embedings.drop(['post_id'], axis=1).sum(axis=1)
post_text_df['EmbMax'] = post_text_embedings.drop(['post_id'], axis=1).max(axis=1)
post_text_df['EmbMin'] = post_text_embedings.drop(['post_id'], axis=1).min(axis=1)

In [27]:
post_text_df = post_text_df.drop(['text', 'embeddings'], axis=1)

In [28]:
post_text_df.head()

,post_id,topic,EmbMean,EmbSum,EmbMax,EmbMin
0,1,business,0.019389,14.890462,10.335843,-0.564470
1,2,business,0.019227,14.766475,10.597654,-0.654997
2,3,business,0.019234,14.771659,10.482660,-0.622263
3,4,business,0.019234,14.771855,10.398684,-0.622589
4,5,business,0.019292,14.816212,10.584318,-0.592012


### Объединим таблицы в одну 


In [29]:
feed_data.head()

,timestamp,user_id,post_id,target,hour,month
0,2021-11-26 15:50:14,200,1881,1,15,11
1,2021-12-13 08:55:58,200,1452,1,8,12
2,2021-11-30 09:37:28,200,249,1,9,11
3,2021-12-13 08:14:54,200,1198,1,8,12
4,2021-11-20 22:45:25,200,6884,1,22,11


In [30]:
post_text_df.head()

,post_id,topic,EmbMean,EmbSum,EmbMax,EmbMin
0,1,business,0.019389,14.890462,10.335843,-0.564470
1,2,business,0.019227,14.766475,10.597654,-0.654997
2,3,business,0.019234,14.771659,10.482660,-0.622263
3,4,business,0.019234,14.771855,10.398684,-0.622589
4,5,business,0.019292,14.816212,10.584318,-0.592012


In [31]:
user_data.head()

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [32]:
feed_data.shape, post_text_df.shape, user_data.shape

((4593428, 6), (7023, 6), (163205, 8))

In [33]:
# объединим таблицы feed_data и post_text_df по post_id
df = pd.merge(feed_data, post_text_df, on='post_id', how='left')


In [34]:
df.head()

,timestamp,user_id,post_id,target,hour,month,topic,EmbMean,EmbSum,EmbMax,EmbMin
0,2021-11-26 15:50:14,200,1881,1,15,11,sport,0.019392,14.893343,10.586618,-0.613332
1,2021-12-13 08:55:58,200,1452,1,8,12,sport,0.019540,15.006635,10.598098,-0.609379
2,2021-11-30 09:37:28,200,249,1,9,11,business,0.019154,14.710227,10.432479,-0.654440
3,2021-12-13 08:14:54,200,1198,1,8,12,politics,0.019333,14.847370,10.571182,-0.605378
4,2021-11-20 22:45:25,200,6884,1,22,11,movie,0.020465,15.717491,11.259039,-0.432301


In [35]:
df.shape

(4593428, 11)

In [36]:
### объединим таблицы df и user_data по user_id


In [37]:
df = pd.merge(df, user_data, on='user_id', how='left')


In [38]:
df.head()

,timestamp,user_id,post_id,target,hour,month,topic,EmbMean,EmbSum,EmbMax,EmbMin,gender,age,country,city,exp_group,os,source
0,2021-11-26 15:50:14,200,1881,1,15,11,sport,0.019392,14.893343,10.586618,-0.613332,1,34,Russia,Degtyarsk,3,Android,ads
1,2021-12-13 08:55:58,200,1452,1,8,12,sport,0.019540,15.006635,10.598098,-0.609379,1,34,Russia,Degtyarsk,3,Android,ads
2,2021-11-30 09:37:28,200,249,1,9,11,business,0.019154,14.710227,10.432479,-0.654440,1,34,Russia,Degtyarsk,3,Android,ads
3,2021-12-13 08:14:54,200,1198,1,8,12,politics,0.019333,14.847370,10.571182,-0.605378,1,34,Russia,Degtyarsk,3,Android,ads
4,2021-11-20 22:45:25,200,6884,1,22,11,movie,0.020465,15.717491,11.259039,-0.432301,1,34,Russia,Degtyarsk,3,Android,ads


In [39]:
df.shape

(4593428, 18)

In [40]:
# отсортируем данные по timestamp_sec

In [41]:
df = df.sort_values('timestamp')
df.head()

,timestamp,user_id,post_id,target,hour,month,topic,EmbMean,EmbSum,EmbMax,EmbMin,gender,age,country,city,exp_group,os,source
1957288,2021-10-01 06:01:40,136194,1205,1,6,10,politics,0.019407,14.904503,10.609164,-0.627050,0,24,Russia,Moscow,3,iOS,organic
2642283,2021-10-01 06:01:40,15471,2810,0,6,10,covid,0.019554,15.017584,11.109459,-0.715483,1,21,Russia,Tambov,4,iOS,ads
3539078,2021-10-01 06:01:40,87173,6030,0,6,10,movie,0.019976,15.341510,10.974415,-0.560105,1,36,Russia,Novokuznetsk,0,Android,ads
2262431,2021-10-01 06:01:40,156604,573,1,6,10,entertainment,0.019497,14.974025,10.697226,-0.668483,1,30,Ukraine,Chernihiv,1,Android,organic
4329879,2021-10-01 06:01:52,148574,1294,0,6,10,politics,0.019067,14.643310,10.555171,-0.682483,1,21,Russia,Novosibirsk,0,iOS,organic


In [42]:
# df.to_csv("df.csv", index=False)

In [43]:
# df = pd.read_csv("df.csv")

### Подготовка данных для обучения

In [44]:
df.head()

,timestamp,user_id,post_id,target,hour,month,topic,EmbMean,EmbSum,EmbMax,EmbMin,gender,age,country,city,exp_group,os,source
1957288,2021-10-01 06:01:40,136194,1205,1,6,10,politics,0.019407,14.904503,10.609164,-0.627050,0,24,Russia,Moscow,3,iOS,organic
2642283,2021-10-01 06:01:40,15471,2810,0,6,10,covid,0.019554,15.017584,11.109459,-0.715483,1,21,Russia,Tambov,4,iOS,ads
3539078,2021-10-01 06:01:40,87173,6030,0,6,10,movie,0.019976,15.341510,10.974415,-0.560105,1,36,Russia,Novokuznetsk,0,Android,ads
2262431,2021-10-01 06:01:40,156604,573,1,6,10,entertainment,0.019497,14.974025,10.697226,-0.668483,1,30,Ukraine,Chernihiv,1,Android,organic
4329879,2021-10-01 06:01:52,148574,1294,0,6,10,politics,0.019067,14.643310,10.555171,-0.682483,1,21,Russia,Novosibirsk,0,iOS,organic


In [45]:
df.shape

(4593428, 18)

train_df = train_test_split(df, test_size=0.03, train_size=0.3, stratify=df['target'], random_state=42)[0].sort_values('timestamp')
#train.to_csv("train.csv", index=False)

In [46]:
df.timestamp.min(), df.timestamp.max(), df.shape[0]

(Timestamp('2021-10-01 06:01:40'), Timestamp('2021-12-29 23:51:06'), 4593428)

In [47]:
### Разделим выборку на train-test

# train = pd.read_csv("train.csv")
train_df = df.iloc[:-int(df.shape[0]/4)].copy()
test_df = df.iloc[-int(df.shape[0]/4):].copy()

In [48]:
train_df.head(2)

,timestamp,user_id,post_id,target,hour,month,topic,EmbMean,EmbSum,EmbMax,EmbMin,gender,age,country,city,exp_group,os,source
1957288,2021-10-01 06:01:40,136194,1205,1,6,10,politics,0.019407,14.904503,10.609164,-0.627050,0,24,Russia,Moscow,3,iOS,organic
2642283,2021-10-01 06:01:40,15471,2810,0,6,10,covid,0.019554,15.017584,11.109459,-0.715483,1,21,Russia,Tambov,4,iOS,ads


In [50]:
# Почистим данные
del user_data
del post_text_embedings
del post_text_df
del feed_data
del df

NameError: name 'user_data' is not defined

In [51]:
from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import TargetEncoder
from sklearn.preprocessing import OneHotEncoder

In [52]:
# Выделим категориальные признаки
cat_columns = ['topic', 'country', 'city', 'os', 'source']

In [53]:
# применим для тестовой и трейн выборки one_hot и mean_target encoding

In [54]:

for col in cat_columns:
    # применим one_hot_encoding для колонок в которых менее 5 уникальных значений
    if train_df[col].nunique() < 5:
        one_hot = pd.get_dummies(train_df[col], prefix=col, drop_first=True)
        train_df = pd.concat((train_df.drop(col, axis=1), one_hot), axis=1)
    # применим mean_target_encoding для колонок в которых менее 5 уникальных значений
    else:
        train_df[col] = train_df[col].map(train_df.groupby(col)['target'].mean())


In [55]:

for col in cat_columns:
    # применим one_hot_encoding для колонок в которых менее 5 уникальных значений
    if test_df[col].nunique() < 5:
        one_hot = pd.get_dummies(test_df[col], prefix=col, drop_first=True)
        test_df = pd.concat((test_df.drop(col, axis=1), one_hot), axis=1)
    # применим mean_target_encoding для колонок в которых менее 5 уникальных значений
    else:
        test_df[col] = test_df[col].map(test_df.groupby(col)['target'].mean())


In [56]:
test_df.shape

(1148357, 18)

In [57]:
# Выделим фичи и таргеты
X_train = train_df.drop(['target', 'timestamp'], axis=1)
X_test = test_df.drop(['target', 'timestamp'], axis=1)

y_train = train_df['target']
y_test = test_df['target']

In [58]:
X_train.head(2)

,user_id,post_id,hour,month,topic,EmbMean,EmbSum,EmbMax,EmbMin,gender,age,country,city,exp_group,os_iOS,source_organic
1957288,136194,1205,6,10,0.497975,0.019407,14.904503,10.609164,-0.627050,0,24,0.50898,0.538609,3,1,1
2642283,15471,2810,6,10,0.514470,0.019554,15.017584,11.109459,-0.715483,1,21,0.50898,0.515477,4,1,0


In [59]:
# df.shape, 
train_df.shape, test_df.shape

((3445071, 18), (1148357, 18))

In [60]:
print(y_train.value_counts())
print(y_test.value_counts())

1    1766279
0    1678792
Name: target, dtype: int64
1    610738
0    537619
Name: target, dtype: int64


In [61]:
X_train.head()

,user_id,post_id,hour,month,topic,EmbMean,EmbSum,EmbMax,EmbMin,gender,age,country,city,exp_group,os_iOS,source_organic
1957288,136194,1205,6,10,0.497975,0.019407,14.904503,10.609164,-0.627050,0,24,0.508980,0.538609,3,1,1
2642283,15471,2810,6,10,0.514470,0.019554,15.017584,11.109459,-0.715483,1,21,0.508980,0.515477,4,1,0
3539078,87173,6030,6,10,0.516163,0.019976,15.341510,10.974415,-0.560105,1,36,0.508980,0.449953,0,0,0
2262431,156604,573,6,10,0.519489,0.019497,14.974025,10.697226,-0.668483,1,30,0.544855,0.565834,1,0,1
4329879,148574,1294,6,10,0.497975,0.019067,14.643310,10.555171,-0.682483,1,21,0.508980,0.474102,0,1,1


### Реализуем контентный подход

### обучим catboost и подберем гиперпараметры 

In [67]:

# Задаем пространство параметров для перебора
param_grid = {'iterations': [100, 200, 500],
              'depth': [3, 5, 10],
              'learning_rate': [0.01, 0.05, 0.1, 0.5]}
# Создаем экземпляр модели CatBoostClassifier
model = CatBoostClassifier()#verbose=False)
# Создаем экземпляр StratifiedKFold для кросс-валидации
cv = StratifiedKFold(n_splits=2, shuffle=False)#, random_state=42, shuffle=False)
# Создаем объект GridSearchCV с заданными параметрами
grid_search = GridSearchCV(model,
                           param_grid,
                           cv=cv,
                           scoring='roc_auc',
                           n_jobs=2
                          )
# Производим поиск по заданной сетке параметров
grid_search.fit(X_train, y_train)
# Выводим наилучшие параметры и значение метрики качества
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)


0:	learn: 0.6783378	total: 1.23s	remaining: 10m 12s
1:	learn: 0.6738312	total: 2.4s	remaining: 9m 58s
2:	learn: 0.6725104	total: 3.54s	remaining: 9m 46s
3:	learn: 0.6718874	total: 4.6s	remaining: 9m 29s
4:	learn: 0.6646637	total: 5.69s	remaining: 9m 23s
5:	learn: 0.6608610	total: 6.81s	remaining: 9m 20s
6:	learn: 0.6604206	total: 7.92s	remaining: 9m 17s
7:	learn: 0.6593924	total: 8.92s	remaining: 9m 8s
8:	learn: 0.6590225	total: 9.88s	remaining: 8m 59s
9:	learn: 0.6581198	total: 10.8s	remaining: 8m 50s
10:	learn: 0.6573429	total: 11.7s	remaining: 8m 42s
11:	learn: 0.6567830	total: 12.7s	remaining: 8m 35s
12:	learn: 0.6566304	total: 13.6s	remaining: 8m 29s
13:	learn: 0.6558047	total: 14.6s	remaining: 8m 26s
14:	learn: 0.6552208	total: 15.6s	remaining: 8m 25s
15:	learn: 0.6548901	total: 16.5s	remaining: 8m 19s
16:	learn: 0.6541579	total: 17.5s	remaining: 8m 16s
17:	learn: 0.6539792	total: 18.4s	remaining: 8m 13s
18:	learn: 0.6537071	total: 19.4s	remaining: 8m 11s
19:	learn: 0.6532805	tot

157:	learn: 0.6374551	total: 2m 31s	remaining: 5m 27s
158:	learn: 0.6374134	total: 2m 32s	remaining: 5m 26s
159:	learn: 0.6373763	total: 2m 33s	remaining: 5m 26s
160:	learn: 0.6373106	total: 2m 34s	remaining: 5m 25s
161:	learn: 0.6372492	total: 2m 35s	remaining: 5m 24s
162:	learn: 0.6371771	total: 2m 36s	remaining: 5m 24s
163:	learn: 0.6371156	total: 2m 37s	remaining: 5m 23s
164:	learn: 0.6370397	total: 2m 38s	remaining: 5m 22s
165:	learn: 0.6369702	total: 2m 39s	remaining: 5m 21s
166:	learn: 0.6369226	total: 2m 40s	remaining: 5m 20s
167:	learn: 0.6368954	total: 2m 42s	remaining: 5m 20s
168:	learn: 0.6367991	total: 2m 43s	remaining: 5m 19s
169:	learn: 0.6367414	total: 2m 44s	remaining: 5m 18s
170:	learn: 0.6366652	total: 2m 45s	remaining: 5m 18s
171:	learn: 0.6366053	total: 2m 46s	remaining: 5m 17s
172:	learn: 0.6365543	total: 2m 47s	remaining: 5m 16s
173:	learn: 0.6364944	total: 2m 48s	remaining: 5m 15s
174:	learn: 0.6364402	total: 2m 49s	remaining: 5m 14s
175:	learn: 0.6363720	total:

310:	learn: 0.6305982	total: 4m 58s	remaining: 3m 1s
311:	learn: 0.6305365	total: 4m 59s	remaining: 3m
312:	learn: 0.6305145	total: 5m	remaining: 2m 59s
313:	learn: 0.6304929	total: 5m	remaining: 2m 58s
314:	learn: 0.6304480	total: 5m 1s	remaining: 2m 57s
315:	learn: 0.6303913	total: 5m 2s	remaining: 2m 56s
316:	learn: 0.6303673	total: 5m 3s	remaining: 2m 55s
317:	learn: 0.6303239	total: 5m 4s	remaining: 2m 54s
318:	learn: 0.6302830	total: 5m 5s	remaining: 2m 53s
319:	learn: 0.6302623	total: 5m 6s	remaining: 2m 52s
320:	learn: 0.6302330	total: 5m 7s	remaining: 2m 51s
321:	learn: 0.6302003	total: 5m 8s	remaining: 2m 50s
322:	learn: 0.6301654	total: 5m 9s	remaining: 2m 49s
323:	learn: 0.6301446	total: 5m 10s	remaining: 2m 48s
324:	learn: 0.6301251	total: 5m 11s	remaining: 2m 47s
325:	learn: 0.6301103	total: 5m 12s	remaining: 2m 47s
326:	learn: 0.6300843	total: 5m 14s	remaining: 2m 46s
327:	learn: 0.6300499	total: 5m 15s	remaining: 2m 45s
328:	learn: 0.6300302	total: 5m 16s	remaining: 2m 

464:	learn: 0.6261850	total: 7m 23s	remaining: 33.4s
465:	learn: 0.6261562	total: 7m 24s	remaining: 32.5s
466:	learn: 0.6261373	total: 7m 25s	remaining: 31.5s
467:	learn: 0.6261256	total: 7m 26s	remaining: 30.5s
468:	learn: 0.6260756	total: 7m 27s	remaining: 29.6s
469:	learn: 0.6260475	total: 7m 28s	remaining: 28.6s
470:	learn: 0.6260167	total: 7m 29s	remaining: 27.7s
471:	learn: 0.6259791	total: 7m 30s	remaining: 26.7s
472:	learn: 0.6259455	total: 7m 31s	remaining: 25.8s
473:	learn: 0.6259144	total: 7m 32s	remaining: 24.8s
474:	learn: 0.6258877	total: 7m 33s	remaining: 23.9s
475:	learn: 0.6258621	total: 7m 34s	remaining: 22.9s
476:	learn: 0.6258417	total: 7m 35s	remaining: 22s
477:	learn: 0.6258237	total: 7m 36s	remaining: 21s
478:	learn: 0.6257847	total: 7m 37s	remaining: 20.1s
479:	learn: 0.6257565	total: 7m 38s	remaining: 19.1s
480:	learn: 0.6257491	total: 7m 39s	remaining: 18.1s
481:	learn: 0.6257087	total: 7m 40s	remaining: 17.2s
482:	learn: 0.6256868	total: 7m 40s	remaining: 16.

In [68]:
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best parameters: {'depth': 5, 'iterations': 500, 'learning_rate': 0.5}
Best score: 0.6810427663135731


In [69]:
from sklearn.metrics import roc_auc_score

print(f"Качество на трейне: {roc_auc_score(y_train, grid_search.predict_proba(X_train)[:, 1])}")
print(f"Качество на тесте: {roc_auc_score(y_test, grid_search.predict_proba(X_test)[:, 1])}")

Качество на трейне: 0.698762037321391
Качество на тесте: 0.7079818875140538


In [ ]:
precision_score( y_test, grid_search.predict(X_test)) 

```
precision_list = []

for l_rate in learning_rates:
    for depth in depth_list:
        for iterations in iterations_list:
            model = CatBoostClassifier(
                                        c 
                                        learning_rate=l_rate,
                                        depth=depth,
                                        iterations=iterations,
                                        cat_features=['city'], 
                                        eval_metric="Precision")

            model.fit(X_train, y_train
                      )

            precision = precision_score( y_test, 
                                         model.predict(X_test)) 
            
            precision_list.append(f'l_rate {l_rate}, depth: {depth}, iteratipns {iterations} ---  precision {precision}') 
              
        
        ```

In [ ]:
precision_list

In [62]:

model = CatBoostClassifier(verbose=False,
                                learning_rate=0.5,
                                depth=5,
                                iterations=1000,
                                )


model

In [63]:
model.fit(X_train, y_train)

In [64]:
from sklearn.metrics import roc_auc_score

print(f"Качество на трейне: {roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])}")
print(f"Качество на тесте: {roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])}")

Качество на трейне: 0.695209531203209
Качество на тесте: 0.6282571618547976


In [65]:

model.save_model('Catboost_model_2_2',
                      format='cbm')

### загрузим модель catboost

In [43]:
from_file_model = CatBoostClassifier()
from_file_model.load_model("Catboost_model_3")

In [46]:
from sklearn.metrics import roc_auc_score

print(f"Качество на трейне: {roc_auc_score(y_train, from_file_model.predict_proba(X_train)[:, 1])}")
print(f"Качество на тесте: {roc_auc_score(y_test, from_file_model.predict_proba(X_test)[:, 1])}")

Качество на трейне: 0.6978275104214423
Качество на тесте: 0.70678475229421


In [ ]:
#Ураа модель работает

In [52]:
from_file_model.get_feature_importance()

array([ 0.78022364, 11.22344425,  2.55119527, 10.85916792,  2.24511844,
        0.67783051,  0.67060052,  1.50641793,  1.82244728,  4.14670781,
        1.54931531,  2.85848842,  2.9850185 ,  3.13211674,  1.85073071,
        1.97572405,  2.08896263,  2.16281571,  1.81578996,  2.31365938,
        2.28459954,  2.17912323,  2.31463334,  2.24133691,  0.21931682,
       25.41898371,  0.20463346,  4.1101068 ,  1.7767518 ,  0.03473941,
        0.        ])

## соберем датасет для выгрузки  в sql

In [66]:
train_df.shape, test_df.shape

((3445071, 18), (1148357, 18))

In [67]:
train_df.head()

,timestamp,user_id,post_id,target,hour,month,topic,EmbMean,EmbSum,EmbMax,EmbMin,gender,age,country,city,exp_group,os_iOS,source_organic
1957288,2021-10-01 06:01:40,136194,1205,1,6,10,0.497975,0.019407,14.904503,10.609164,-0.627050,0,24,0.508980,0.538609,3,1,1
2642283,2021-10-01 06:01:40,15471,2810,0,6,10,0.514470,0.019554,15.017584,11.109459,-0.715483,1,21,0.508980,0.515477,4,1,0
3539078,2021-10-01 06:01:40,87173,6030,0,6,10,0.516163,0.019976,15.341510,10.974415,-0.560105,1,36,0.508980,0.449953,0,0,0
2262431,2021-10-01 06:01:40,156604,573,1,6,10,0.519489,0.019497,14.974025,10.697226,-0.668483,1,30,0.544855,0.565834,1,0,1
4329879,2021-10-01 06:01:52,148574,1294,0,6,10,0.497975,0.019067,14.643310,10.555171,-0.682483,1,21,0.508980,0.474102,0,1,1


In [68]:
df_to_sql_mod_2_2 = pd.concat([train_df, test_df], axis=0)
df_to_sql_mod_2_2

,timestamp,user_id,post_id,target,hour,month,topic,EmbMean,EmbSum,EmbMax,EmbMin,gender,age,country,city,exp_group,os_iOS,source_organic
1957288,2021-10-01 06:01:40,136194,1205,1,6,10,0.497975,0.019407,14.904503,10.609164,-0.627050,0,24,0.508980,0.538609,3,1,1
2642283,2021-10-01 06:01:40,15471,2810,0,6,10,0.514470,0.019554,15.017584,11.109459,-0.715483,1,21,0.508980,0.515477,4,1,0
3539078,2021-10-01 06:01:40,87173,6030,0,6,10,0.516163,0.019976,15.341510,10.974415,-0.560105,1,36,0.508980,0.449953,0,0,0
2262431,2021-10-01 06:01:40,156604,573,1,6,10,0.519489,0.019497,14.974025,10.697226,-0.668483,1,30,0.544855,0.565834,1,0,1
4329879,2021-10-01 06:01:52,148574,1294,0,6,10,0.497975,0.019067,14.643310,10.555171,-0.682483,1,21,0.508980,0.474102,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2524513,2021-12-29 23:45:27,6545,5301,0,23,12,0.535941,0.020557,15.787940,11.266890,-0.474848,1,16,0.553575,0.444444,4,1,0
3331870,2021-12-29 23:48:26,71338,786,0,23,12,0.545044,0.019311,14.830672,10.676378,-0.710154,0,15,0.529694,0.512824,3,0,0
2524512,2021-12-29 23:48:26,6545,5104,0,23,12,0.535941,0.020147,15.472726,11.052585,-0.392648,1,16,0.553575,0.444444,4,1,0
2524511,2021-12-29 23:51:06,6545,2735,0,23,12,0.533241,0.019558,15.020369,10.919322,-0.682631,1,16,0.553575,0.444444,4,1,0
